# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
import ames # The module for loading external data - Ames Housing dataset
import os
import pandas as pd
import json
import ast
import pickle

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment, Model, Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-153813
aml-quickstarts-153813
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Try to load the dataset from the workspace. Otherwise, load if from Kaggle
found = False
ds_key = 'Ames-housing-dataset'
ds_desc = 'Ames Housing training data.'

if ds_key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[ds_key]
    print(f'Found registered {ds_key}, use it.')
    
if not found:
    train, test = ames.load_data_clean()
    print(f"train.shape = {train.shape}, test.shape = {test.shape}")
    # Register the train dataset
    blob = ws.get_default_datastore()
    dataset = TabularDatasetFactory.register_pandas_dataframe(train, blob, name=ds_key, description=ds_desc)

Found registered Ames-housing-dataset, use it.


In [23]:
%%writefile train_xgb.py
"""Train, evaluate and log metrics for selected ML algorithm 
in the Azure workspace context."""

import argparse
import os
import numpy as np
import pandas as pd
import joblib
import ames

from azureml.core.run import Run
from azureml.core import Workspace

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


ws = Workspace.from_config()
ds_key = 'Ames-housing-dataset'
dataset = ws.datasets[ds_key]

train = dataset.to_pandas_dataframe()

X_train, X_test = train_test_split(ames.label_encode(ames.encode_dtypes(train)))
y_train = X_train.pop('SalePrice')
y_test = X_test.pop('SalePrice')

print(f"X_train.shape = {X_train.shape}, X_test.shape = {X_test.shape}")

run = Run.get_context()

model = XGBRegressor(learning_rate=0.1, gamma=2, max_depth=3, objective='reg:squarederror')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

run.log("r2_score", np.float(r2))
print(f'Writting r2 score = {r2} into a log.')

os.makedirs('./outputs', exist_ok=True)
joblib.dump(model, './outputs/model.joblib')

Overwriting train_xgb.py


In [45]:
%%writefile conda_env.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults==1.32.0
- scikit-learn
- xgboost

Overwriting conda_env.yml


In [46]:
# Define an Azure ML environment
# Dependencies are the same as for AutoML experiment
env = Environment.from_conda_specification(name='env', file_path='conda_env.yml')

# Configure the training job
src = ScriptRunConfig(source_directory=".",
                     script='train_xgb.py',
                     compute_target=cpu_cluster,
                     environment=env)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [42]:
# Choose a name for an experiment
experiment_name = 'Ames-housing-hdr'

experiment=Experiment(ws, experiment_name)

In [47]:
run = experiment.submit(src)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here?

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [48]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: Ames-housing-hdr_1628254001_2fc22db9
Web View: https://ml.azure.com/runs/Ames-housing-hdr_1628254001_2fc22db9?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-153813/workspaces/quick-starts-ws-153813&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/20_image_build_log.txt

2021/08/06 12:46:46 Downloading source code...
2021/08/06 12:46:47 Finished downloading source code
2021/08/06 12:46:47 Creating Docker network: acb_default_network, driver: 'bridge'
2021/08/06 12:46:48 Successfully set up Docker network: acb_default_network
2021/08/06 12:46:48 Setting up Docker configuration...
2021/08/06 12:46:48 Successfully set up Docker configuration
2021/08/06 12:46:48 Logging in to registry: 08df9bce870240588e0846fbe2b40525.azurecr.io
2021/08/06 12:46:49 Successfully logged into 08df9bce870240588e0846fbe2b40525.azurecr.io
2021/08/06 12:46:49 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_defau

Executing transaction: ...working... 

    Installed package of scikit-learn can be accelerated using scikit-learn-intelex.
    More details are available here: https://intel.github.io/scikit-learn-intelex

    For example:

        $ conda install scikit-learn-intelex
        $ python -m sklearnex my_application.py

    

done
Installing pip dependencies: ...working... 
Ran pip subprocess with arguments:
['/azureml-envs/azureml_73574990bfa9d423abf70d8a8c925fc1/bin/python', '-m', 'pip', 'install', '-U', '-r', '/azureml-environment-setup/condaenv.1lvc4l3p.requirements.txt']
Pip subprocess output:
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=b98e12da3dfe59604406cea79f8dc5fffb0d2c0d901fcc5a20334eb127677161
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10502 sha256=de9ef8461fe521745c9a405ff90d97c2b6983b9d

Removing intermediate container 2e91e4cc2b5b
 ---> c39e964d6b57
Step 9/18 : ENV PATH /azureml-envs/azureml_73574990bfa9d423abf70d8a8c925fc1/bin:$PATH
 ---> Running in 28cf5da46d39
Removing intermediate container 28cf5da46d39
 ---> 070d47827a4a
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> fad57482d442
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> 68ec3095f13e
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_73574990bfa9d423abf70d8a8c925fc1
 ---> Running in dbb383869796
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with materialized conda environment details
Successfully sent materialized environment details
Removing intermediate container dbb383869796
 ---> d58bc82b6d4b
Step 13/18 : ENV AZUREML


Execution Summary
RunId: Ames-housing-hdr_1628254001_2fc22db9
Web View: https://ml.azure.com/runs/Ames-housing-hdr_1628254001_2fc22db9?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-153813/workspaces/quick-starts-ws-153813&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'Ames-housing-hdr_1628254001_2fc22db9',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-06T12:51:39.43164Z',
 'endTimeUtc': '2021-08-06T13:06:27.60232Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '4bc9a0a7-ac84-4613-a72b-e6107389584b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.RuntimeType': 'Hosttools'},
 'inputDatasets': [{'dataset': {'id': '9dcb2684-b597-40f4-a301-50963b0ee143'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_xgb.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priori

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service